In [1]:
# Convolutional neural network
# Made up of neurons
# Applies convolutional filters

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [3]:
#Device, gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#Hyper params
num_epochs = 2
batch_size = 1
learning_rate = 0.001

cuda


In [4]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_datasets = torchvision.datasets.MNIST(root='./data', train=True,transform=transforms.ToTensor(), download=True)
test_datasets = torchvision.datasets.MNIST(root='./data', train=False,transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_datasets, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_datasets, batch_size=batch_size, shuffle=False)

classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

  0%|          | 0/9912422 [00:00<?, ?it/s]

9913344it [00:00, 11414068.16it/s]                             


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw


29696it [00:00, 29691073.08it/s]         
  0%|          | 0/1648877 [00:00<?, ?it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



1649664it [00:00, 9205803.41it/s]                            
5120it [00:00, ?it/s]                   

Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw




C:\Users\Morten\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        #Colors: 3, output: 6, kernel_size: 5
        self.conv1 = nn.Conv2d(1,6,5)
        #kernel_size: 2, stride: 2
        self.pool = nn.MaxPool2d(2)
        #Input = conv1 output som er 6
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(64*2*2, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64*2*2)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = ConvNet().to(device)

In [6]:
#Loss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [7]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        #forward pass
        outputs = model(images)
        loss = criterion(outputs,labels)

        #backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if(i+1) % 10000 == 0:
            print(f'epoch {epoch+1} / {num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')

print('Finished Training')

C:\Users\Morten\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch 1 / 2, step 10000/60000, loss = 1.1327
epoch 1 / 2, step 20000/60000, loss = 0.1995
epoch 1 / 2, step 30000/60000, loss = 0.1608
epoch 1 / 2, step 40000/60000, loss = 0.0024
epoch 1 / 2, step 50000/60000, loss = 0.0001
epoch 1 / 2, step 60000/60000, loss = 0.0005
epoch 2 / 2, step 10000/60000, loss = 0.0002
epoch 2 / 2, step 20000/60000, loss = 0.0000
epoch 2 / 2, step 30000/60000, loss = 0.0064
epoch 2 / 2, step 40000/60000, loss = 0.0003
epoch 2 / 2, step 50000/60000, loss = 0.0001
epoch 2 / 2, step 60000/60000, loss = 0.0191
Finished Training


In [8]:
#test
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs,1)
        n_samples += labels.shape[0]
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if(label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

acc = 100.0 *n_correct/n_samples
print(f'Accuracy = {acc}')

for i in range(10):
    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
    print(f'Accuracy of {classes[i]}: {acc} %')


Accuracy = 97.74
Accuracy of 0: 99.59183673469387 %
Accuracy of 1: 99.8237885462555 %
Accuracy of 2: 97.67441860465117 %
Accuracy of 3: 97.72277227722772 %
Accuracy of 4: 98.67617107942974 %
Accuracy of 5: 95.96412556053812 %
Accuracy of 6: 97.07724425887265 %
Accuracy of 7: 97.56809338521401 %
Accuracy of 8: 97.12525667351129 %
Accuracy of 9: 95.73835480673935 %
